# To be run once to setup modules and paths


In [ ]:
#Reload modules without shutting notebook
%load_ext autoreload
%autoreload 2

import os
import shutil
import glob
import numpy as np
import h5py
import pickle
from datetime import date
import requests
from IPython.display import clear_output

# Do not show warnings
import warnings
warnings.filterwarnings('ignore')

# Global variables including folders
home = os.path.expanduser("~")
basefolder = os.path.join(home, "my_shared_data_folder")
s1ardfolder = os.path.join(basefolder, "s1ard")
datasets = os.path.join(basefolder, "datasets")

# Upland burn modules
import sys
sys.path.append(os.path.join(home, "notebooks/utils"))
import extract_aoi as extract
import plot_data as plotd
import get_configuration as getc
import portal_credentials as portalc
import call_cophub as callc
import onda_archive as onda
import syscmds as sysc
import run_gpt as rgpt

## setup and clear out temp folder
tmpfolder = os.path.join(home, "temp")
if not os.path.exists(tmpfolder):
    os.mkdir(tmpfolder)
#else:
#    shutil.rmtree(tmpfolder)
#    os.mkdir(tmpfolder)

# Get configuration information
cstudy = "skye" 
#cstudy = "cairngorms" 
#cstudy = "pdistrict" 

basefolder, s1ardfolder, datasets, outfolder, tmpfolder, ofiles, hdfile, pfile, verbose, graphics = getc.get_configuration(cstudy)
if cstudy == "skye":
    aoi = os.path.join(datasets, "Skye_extent_OSGB36.geojson")
elif cstudy == "cairngorms":
    aoi = os.path.join(datasets, "Cairngorms_extent_OSGB36.geojson")
else:
    aoi = os.path.join(datasets, "PDistrict_extent_OSGB36.geojson")
fstart = os.path.basename(aoi).split("_")[0]

# Load Copernicus hub credentials
pfile = "cophub.txt"
home = os.path.expanduser("~")
if not os.path.exists(os.path.join(home, pfile)):
    portalc.save_credentials(pfile)
cop_credentials = portalc.read_credentials(pfile)

# Define date range and search Copernicus Hub

In [ ]:
start_date = date(2019, 1, 1)
end_date = date(2019, 1, 4)

In [ ]:
# Call copernicus hub to get list of products
products, wkt = callc.call_cophub(cop_credentials, start_date, end_date, aoi, SLC = False)
print("Found {} products".format(len(products)))

# Download Copernicus Sentinel-1 GRD data

In [ ]:
from sentinelsat import SentinelAPI
# Log into Copernicus Hub
api = SentinelAPI(cop_credentials[0], cop_credentials[1], 'https://scihub.copernicus.eu/dhus')

print("Downloading GRD files")
s1grdfolder = os.path.join(basefolder, "s1grd") 
grdfiles = []
for download_num, (id1, data) in enumerate(products.iterrows()):
    filename = data['title']
    s1file = os.path.join(s1grdfolder, filename + '.zip')
    splits = filename.split("_")
    dstring = splits[4]
    dstring2 = splits[5]
    cedafile = splits[0] + "_" + dstring[0:8] + "_*_" + dstring[9:15] + "_" + dstring2[9:15] + "_*.tif"
    test = glob.glob(os.path.join(s1ardfolder, cedafile))
    # Check if already stored
    if (len(test) > 0) or os.path.exists(s1file):
        print("{} already downloaded".format(filename))
        grdfiles.append(s1file)
    else:
        info = api.get_product_odata(id1)
        url = info['url'] + '/Online/$value'
        cmd = 'curl -u ' + cop_credentials[0] + ':' + cop_credentials[1] + ' "' + url + '"'
        status = sysc.execmd(cmd, verb=False)   
        if "false</Online>" in status.decode(encoding='UTF-8'): # offline
            if download_num == 0:
                # Load ONDA credentials
                pfile = "onda.txt"
                home = os.path.expanduser("~")
                if not os.path.exists(os.path.join(home, pfile)):
                    portalc.save_credentials(pfile)
                onda_credentials = portalc.read_credentials(pfile)

            # Try to download all products from ONDA if archive file
            onda.batch_download_from_archive(onda_credentials, products, s1grdfolder)
            
            if os.path.exists(s1file):
                print("{} downloaded".format(filename))
                grdfiles.append(s1file)
            
        else: # Try from Copernicus hub
            print("Downloading from Copernicus Hub")
            api.download(id1, directory_path=s1grdfolder)
            
            if os.path.exists(s1file):
                print("{} downloaded".format(filename))
                grdfiles.append(s1file)



# Processing GRD data

In [ ]:
# Process the individual subswaths
import subprocess
xml_folder = os.path.join(home, "notebooks/xml-files")
calib_folder = os.path.join(s1grdfolder, "calibrate")
if not os.path.exists(calib_folder):
    os.mkdir(calib_folder)

count = 0
calfiles = []
for infile in grdfiles:
    basestem = os.path.splitext(os.path.basename(infile))[0]
    print("Running for {}".format(basestem))

    # Setup properties and xml file for calibration step
    xml_file = os.path.join(xml_folder, 'Calibrate.xml')
    properties_file = xml_file.replace(".xml",".properties")

    calibfile = os.path.join(calib_folder, basestem + "_calibrate.dim")
    if os.path.exists(calibfile):
        print("{} processed to calibration stage".format(calibfile))

    else:    
        rgpt.run_gpt(infile,calibfile,xml_file,properties_file)
    count += 1
    calfiles.append(basestem)
    if count == 2:
        break  
        
temporal = True
if not temporal:
    for basestem in calfiles:
        # Setup properties and xml file for terrain correction and filtering step, including subsetting to aoi, using single Lee
        xml_file = os.path.join(xml_folder, 'Terrain-Subset.xml')
        properties_file = xml_file.replace(".xml",".properties")

        terrfile = os.path.join(calib_folder, basestem + "_terrain.dim")
        if os.path.exists(terrfile):
            print("{} processed to terrain stage".format(terrfile))

        else:    
            rgpt.run_gpt(calibfile,terrfile,xml_file,properties_file, subset = wkt)

        # Setup properties and xml file for reprojection and conversion to GeoTIFF
        xml_file = os.path.join(xml_folder, 'Reproject.xml')
        properties_file = xml_file.replace(".xml",".properties")

        outfile = os.path.join(calib_folder, basestem + ".tif")
        if os.path.exists(outfile):
            print("{} processed to final stage".format(outfile))
        else:    
            rgpt.run_gpt(terrfile,outfile,xml_file,properties_file)
else:
    # Setup properties and xml file for terrain correction and filtering step, including subsetting to aoi, using multitemporal
    xml_file = os.path.join(xml_folder, 'Terrain-Subset-Collocate.xml')
    properties_file = xml_file.replace(".xml",".properties")
    calibfile = os.path.join(calib_folder, calfiles[0] + "_calibrate.dim")
    calibfile2 = os.path.join(calib_folder, calfiles[1] + "_calibrate.dim")

    terrfile = os.path.join(calib_folder, calfiles[0] + "_terrain_2.dim")
    if os.path.exists(terrfile):
        print("{} processed to terrain stage".format(terrfile))
    else:    
        rgpt.run_gpt(calibfile,terrfile,xml_file,properties_file, subset = wkt, collocate = calibfile2)

    # Setup properties and xml file for reprojection and conversion to GeoTIFF
    xml_file = os.path.join(xml_folder, 'Reproject.xml')
    properties_file = xml_file.replace(".xml",".properties")

    outfile = os.path.join(calib_folder, basestem + ".tif")
    if os.path.exists(outfile):
        print("{} processed to final stage".format(outfile))
    else:    
        rgpt.run_gpt(terrfile,outfile,xml_file,properties_file)
    


# Download Sentinel-1 ARD data for specific date range

In [ ]:
# Download ARD data
for download_num, (id1, data) in enumerate(products.iterrows()):
    filename = data['title']
    s1file = filename + '.tif'
    splits = filename.split("_")
    dstring = splits[4]
    dstring2 = splits[5]
    cedafile = splits[0] + "_" + dstring[0:8] + "_*_" + dstring[9:15] + "_" + dstring2[9:15] + "_*.tif"
    test = glob.glob(os.path.join(s1ardfolder, cedafile))
    # Check if already stored
    if (len(test) > 0) or os.path.exists(s1file):
        print("{} already downloaded".format(filename))
    else:
        if "Skye" in aoi or "Cairngorms" in aoi: # Download data from JNCC FTP

            # Load CEDA FTP credentials
            pfile = "ceda.txt"
            home = os.path.expanduser("~")
            if not os.path.exists(os.path.join(home, pfile)):
                portalc.save_credentials(pfile)
            ceda_credentials = portalc.read_credentials(pfile)

            print("Querying CEDA FTP")
            ceda = "ftp://ftp.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/"

            # Run curl command to try to download
            print("Trying to download {} ".format(filename))
            urlpath = ceda + os.path.join(dstring[0:4],os.path.join(dstring[4:6], dstring[6:8]))
            cmd = 'wget --user ' + ceda_credentials[0] + ' --password ' + ceda_credentials[1] + ' -P ' + s1ardfolder
            cmd += ' -r --no-directories --no-host-directories --no-parent -A "' + cedafile + '" ' + urlpath
            print("CMD: ",cmd)
            output = sysc.execmd(cmd, verb = False)
            if output != None:
                size = len(output)
                print(output[:size - 30]) # print last 30 characters

        elif "PDistrict" in aoi:

            # Load Defra credentials
            pfile = "defra_api.txt"
            home = os.path.expanduser("~")
            if not os.path.exists(os.path.join(home, pfile)):
                portalc.save_credentials(pfile)
            defra_credentials = portalc.read_credentials(pfile)

            print("Querying Defra EO Data Service ")
            date_value = dstring[0:4] + '-' + dstring[4:6] + '-' + dstring[6:8]
            urlpath = 'https://earthobs.defra.gov.uk' + ('/api/layers/?username=' + defra_credentials[0] 
                                    + '&api_key=' + defra_credentials[1]
                                    + '&limit=20000&offset=0'
                                    + '&geometry="' + wkt.replace(" ","+") + '"'
                                    + '&date__range=' + date_value + '%2000:00,' + date_value + '%2023:59'
                                    + '&title__icontains=' + splits[0]
                                    + '&type__in=raster'
                                   )
            # Making a get request 
            response = requests.get(urlpath)
            print("Status: {} {}".format(response.status_code, response.request))
            if response.status_code != 200:
                print("Failed to download: ", response)
                break


## Import the Sentinel-1 ARD data for the Area of Interest (AOI)

In [ ]:
# Import list of Sentinel-1 ARD (backscatter files)
s1ards = glob.glob(os.path.join(s1ardfolder, "*.tif"))
numfiles  = len(s1ards)
print("Found {} Sentinel-1 ARD files".format(numfiles))

In [ ]:
# Subset each file to the area of interest
ofiles = []
print("xmin xmax ymin ymax 109880.178605,169814.04887,808556.85961,881457.683008")
if not os.path.exists(aoi):
    print("Could not find {}".format(aoi))
else:
    for ifile in s1ards:
        ofile = os.path.join(tmpfolder, os.path.basename(ifile))
        
        # Delete if exists
        #if os.path.exists(ofile):
        #    os.remove(ofile)
        # Skip if exists
        if os.path.exists(ofile):
            ofiles.append(ofile)
            print("{} exists, skipping".format(os.path.basename(ofile)))
        else:
            # Create subset in temp folder
            extract.cut_by_geojson(ifile, ofile, aoi, verb = True)

            if os.path.exists(ofile):
                ofiles.append(ofile)
                print("Generated: ",ofile)

print("Finished processing, {} output subsets available".format(len(ofiles)))

# Check for same-day files and merge if they exist

In [ ]:
import sys
sys.path.append('/usr/bin')
import gdal_merge as gm

# Sort files by names
ofiles = sorted(ofiles,key=lambda x: int(os.path.splitext(os.path.basename(x))[0][4:12]))

subfiles = []
pdatestr = ""
for ofile in ofiles:
    dir, fname = os.path.split(ofile)
    fnames = fname.split("_")
    datestr = fnames[1]
    if datestr == pdatestr:
        dir, fname2 = os.path.split(pfile)
        if int(fname[4]) < int(fname2[4]):
            mfile = os.path.join(dir, fname[:26] + fname2[26:])
        else:
            mfile = os.path.join(dir, fname2[:26] + fname[26:])
        if not os.path.exists(mfile):
            print("Merging {} and {} to create {}".format(pfile, ofile, mfile))
            shutil.copyfile(ofile,mfile)
            gm.main(['', '-init', '0', '-o', mfile, pfile, ofile])
            if os.path.exists(mfile):
                os.remove(pfile)
                os.remove(ofile)        
                subfiles.append(mfile)
            else:
                print("Failed to generate merged file {}".format(mfile))
        else:
            print("Merged file {} already exists".format(mfile))
            
    else:
        pdatestr = datestr
        pfile = ofile
        subfiles.append(ofile)
# Create final list of files
ofiles = []
for file in subfiles:
    if os.path.exists(file):
        ofiles.append(file)
print("Finished processing, {} output subsets available".format(len(ofiles)))

# Display generated subset

In [ ]:
from osgeo import ogr, osr, gdal
# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Setup HDF file to store full sized darray
hdfile = os.path.join(tmpfolder, 'skye_s1ard.h5')
pfile = os.path.join(tmpfolder, 'skye_s1ard.txt')
nfile = os.path.join(tmpfolder, 'skye_s1ard.npy')
overwrite = False

if overwrite or not os.path.exists(hdfile):
    
    # Delete existing file
    os.remove(hdfile)
    
    # Setup
    darray = 0
    layers = 0

    with h5py.File(hdfile, 'w') as hf_object:
        for ofile in ofiles:
            # Open original data as read only
            ds = gdal.Open(ofile, gdal.GA_ReadOnly)
            bands = ds.RasterCount
            print("Loading {} into HDF file".format(os.path.basename(ofile)))
            for band in range(bands):
                image = ds.GetRasterBand(band+1).ReadAsArray()
                if band == 0:
                    xdim, ydim = image.shape

                    # Getting georeference info
                    transform = ds.GetGeoTransform()
                    projection = ds.GetProjection()
                    xOrigin = transform[0] # top left x 
                    yOrigin = transform[3] # top left y 
                    pixelWidth = transform[1] # w-e pixel resolution 
                    pixelHeight = -transform[5] # n-s pixel resolution (negative value) 

                if not isinstance(darray, np.ndarray):  # Not an array, setup output data array
                    print("Setting up darray")
                    #darray = np.zeros((xdim+1, ydim+1, bands, len(ofiles)), dtype=np.float)
                    darray = np.zeros((1, bands, xdim+1, ydim+1), dtype=np.float64)
                    print("darray shape: ",darray.shape)

                darray[0, band, 0:image.shape[0],0:image.shape[1]] = image[0:image.shape[0],0:image.shape[1]]

                dir, fname = os.path.split(ofile)
                print("{} Layer {} Band {} min {:.3f} max {:.3f}".format(fname[4:12],layers,band,np.nanmin(image),np.nanmax(image)))

            if layers == 0:
                # First write to HDF file
                hf_object.create_dataset('s1ard', data=darray, compression="gzip", chunks=(1, bands, xdim+1, ydim+1), maxshape=(None, bands, xdim+1, ydim+1)) 
            else:
                # Append to HDF file
                hf_object["s1ard"].resize((hf_object["s1ard"].shape[0] + darray.shape[0]), axis = 0)
                hf_object["s1ard"][-darray.shape[0]:] = darray

            # Increment layer count    
            layers += 1
            ds = None

    del darray
    # Close HDF file
    hf_object.close()

    print("Created stack of data in HDF file: {}".format(hdfile))
else:
    print("HDF file {} exists".format(hdfile))
    

if overwrite or not os.path.exists(pfile):
    # Create pickle file to store metadata
    layer = 0
    # Setup dictionary
    data_dict = []
    for file in ofiles:
        splits = os.path.basename(file).split("_")
        data_dict.append({'layer': layer, 'sensor': splits[0], 'date': splits[1], 'stime': splits[4], 'etime': splits[5], 
                          'polarisation': splits[6], 'rorbit': splits[2], 'direction': splits[3]})
        layer += 1

    # Write pickle file
    outfile = open(pfile,'wb')
    pickle.dump(data_dict,outfile)
    outfile.close()
    print("Wrote pickle file to: {}".format(pfile))
    del data_dict
else:
    print("Pickle file {} exists".format(pfile))
        
if overwrite or not os.path.exists(nfile):
    # Create file to save list of filenames
    np.save(nfile, ofiles)
    print("Wrote list file to: {}".format(nfile))
else:
    print("List file {} exists".format(nfile))
    
# Deploy Skye subset to shared folder
skyefolder = os.path.join(basefolder, "skye")
if not os.path.exists(skyefolder):
    os.mkdir(skyefolder)
new_hdfile = os.path.join(skyefolder, 'skye_s1ard.h5')
new_pfile = os.path.join(skyefolder, 'skye_s1ard.txt')
new_nfile = os.path.join(skyefolder, 'skye_s1ard.npy')
if overwrite or not os.path.exists(new_hdfile):
    shutil.copy(hdfile, new_hdfile)
if overwrite or not os.path.exists(new_pfile):
    shutil.copy(pfile, new_pfile)
if overwrite or not os.path.exists(new_nfile):
    shutil.copy(nfile, new_nfile)

tfiles = ofiles
ofiles = []
for file in tfiles:
        new_file = os.path.join(skyefolder, os.path.basename(file))
        if overwrite or not os.path.exists(new_file):
            shutil.copy(file, new_file)
        ofiles.append(os.path.basename(file))
        
print("Deployed to {}".format(skyefolder))

In [ ]:
# Read hdf file and display chosen band: VV or VH
hf_object = h5py.File(hdfile, 'r')
d = hf_object['s1ard']
print("Displaying {} layers".format(len(ofiles)))
plotd.plot_images(d, ofiles, verb = False, hdf = 'VV')
hf_object.close()

In [ ]:
# Write mask array
subfile = os.path.join(tmpfolder, "subfile.tif")
if os.path.exists(subfile):
    os.remove(subfile)
driver = gdal.GetDriverByName('GTiff')
ds = driver.Create(subfile, subxdim, subydim, 1, gdal.GDT_Float32)
ds.SetGeoTransform(new_transform)
ds.SetProjection(projection)
ds.GetRasterBand(1).SetNoDataValue(0)
#ds.GetRasterBand(1).WriteArray(sdata)
ds = None